# The quadradic-program solver ProxQP

This appendix is a very brief introduction to the solver ProxQP, dedicated to problems with quadric cost and linear (inequality and equality) constraints. We will need numpy and the ProxSuite package.

In [ ]:
import proxsuite
import numpy as np
from numpy.linalg import eig
import random

## Problem definition 
We consider the following linearly constrained quadratic program:
$$\min_x \frac{1}{2} x^T H x - g^T x$$
Under the constraints:
$$\underline{b} \le x \le \overline{b}$$
$$A_e x = b_e$$
$$\underline{b}_i \le A_i x \le \overline{b}_i$$


where $H$ is the cost Hessian, $g$ the cost gradient, $\underline{b}$,$\overline{b}$ the bounds (box constraints), $A_e$,$b_e$ define the equality constraints and $A_i,\underline{b}_i,\overline{b}_i$ the inequality constraints.

## Random construction

Here is a piece of code to define a problem, with the following sizes:

In [ ]:
# %load tp4/generated/example_proxqp_param
# ### TEST PARAMETERS
NX = 20 # x dimension (search space)
NEQ = 5 # number of equalities
NINEQ = 3 # Number of inequalities
WITH_BOUNDS = True # Additional bounds on x
VERBOSE = False # Do you want to see the result?
ACCURACY = 1e-6 # Threshold for solver stoping criteria and posterior checks


Sampling is relatively straightforward. You just have to pay attention to the Hessian that should be symmetrical positive definite, and the bounds that should be sorted.

In [ ]:
# %load tp4/generated/example_proxqp_matrices
# Cost
H = np.random.rand(NX, NX) * 2 - 1
H = H @ H.T  ### Make it positive symmetric
g = np.random.rand(NX)

Ae = np.random.rand(NEQ, NX)*2-1
be = np.random.rand(NEQ)*2-1

Ai = np.random.rand(NINEQ, NX)*2-1
bi = np.sort(np.random.rand(NINEQ,2)*2-1,1)
for i in range(NINEQ):
    # Half inequalities are double bounds
    # One quarter are pure lower
    # One quarter are pure upper
    r = random.randint(0,3)
    if r==0: bi[i,0] = -1e20
    elif r==1: bi[i,1] = 1e20

bounds = np.sort(np.random.rand(NX,2)*2-1,1)+[-1,1]
for i in range(NX):
    # Half inequalities are double bounds
    # One quarter are pure lower
    # One quarter are pure upper
    r = random.randint(0,3)
    if r==0: bounds[i,0] = -1e20
    elif r==1: bounds[i,1] = 1e20


## Solver call

The solver must first be initialized to the matrix sizes, then call.

In [ ]:
# %load tp4/generated/example_proxqp_solve
#[x, cost, _, niter, lag, iact] =
qp = proxsuite.proxqp.dense.QP(NX, NEQ, NINEQ, WITH_BOUNDS)
qp.settings.eps_abs = ACCURACY/1e3
qp.init(H, g, Ae, be, Ai, bi[:,0], bi[:,1],
        bounds[:,0] if WITH_BOUNDS else None,
        bounds[:,1] if WITH_BOUNDS else None)
qp.solve()


The results are exported through the *results* field. The solver provides the primal variable $x^*$, and the multipliers of the equality $y^*$ and inequality $z^*$ constraints (bounds and inequalities are put together in $z^*$).

In [ ]:
# %load tp4/generated/example_proxqp_result
x,y,z = qp.results.x,qp.results.y,qp.results.z
if WITH_BOUNDS:
    w = z[NINEQ:] # bounds
    z = z[:NINEQ] # general inequalities
cost = qp.results.info.objValue


Let's print the result to finish!

In [ ]:
# %load tp4/generated/example_proxqp_print
print("Primal optimum x: {}".format(x))
print("Dual optimum (equalities) y: {}".format(y))
print("Dual optimum (ineq) z: {}".format(z))
print("Dual optimum (bounds) w: {}".format(w))
